This function computes the right hand side of the system of ODEs:

\begin{align}
\dfrac{d}{ds}\mathbf{x}(s) &= \mathbf{e}_i = \begin{pmatrix} \cos(\phi(s)) \\ \sin(\phi(s)) \end{pmatrix} \\
\dfrac{d}{ds}\phi(s) &= -\dfrac{1}{2s_k}\mathbf{e}_k^T(\nabla_x S \mathbf{e}_i) \mathbf{e}_i
\end{align}, where $ \mathbf{e}_k = \dfrac{d}{ds} \mathbf{e}_i = \begin{pmatrix} -\sin(\phi(s)) \\ \cos(\phi(s)) \end{pmatrix} $.

$ \dfrac{d}{ds}\phi(s) $ depends on both the spatial coordinates $ \mathbf{x} = (x, y) $ but also on the angle $ \phi $.

In [1]:
import sys, os

# get current directory
path = os.getcwd()

# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])

# add utils folder to current working path
sys.path.append(parent_directory+"/subfunctions")

In [ ]:
# Import math tools
from math import cos, sin, sqrt, pi

# Import function which checks if trajectory is in a domain where the velocity field is still defined
from ipynb.fs.defs.check_location import check_location

In [ ]:
def _tensorline_equation(t, x_phi, interp_phi_prime, s_threshold, interp_s, X, Y, defined_domain):
    
    '''
    
    Computes tensorlines based on the Fast Tensorline Computation (FTC) algorithm.
    
    Parameters:
        t:                dummy time-variable (not used in this function but required for ODE-solver)
        x_phi:            list, 
                          x_phi[0] contains x-position
                          x_phi[1] contains y-position
                          x_phi[2] contains angles
        interp_phi_prime: Interpolant-object for phi_prime
        s_threshold:      float, threshold that guarantees that trajectory is sufficiently far away from singularity
        interp_s:         Interpolant-object for evaluating attraction/repulsion rate
        X:                array(Ny, Nx): X-meshgrid
        Y:                array(Ny, Nx): Y-meshgrid
        defined_domain:   array(Ny, Nx): meshgrid defining domain where velocity field is defined, if 1 -->velocity field is defined, if 0 --> velocity field is not defined (e.g. land, walls, etc...)
    
    Returns:
    
        RHS:              list, Normalized right-hand-side of differential equation. If point is close to singularity, then [0,0,0] is returned and integration will stop.
    '''
    
    # extract x, y, phi
    x, y, phi = x_phi[0], x_phi[1], x_phi[2]%(2*pi)
    
    # check particle location
    if check_location(X, Y, defined_domain, x_phi[:2], True)[0] == "IN":
        
        # make sure that trajectory is sufficiently far away from singularity (=rate of attraction\repulsion is sufficiently high)
        if abs(interp_s(y, x)[0][0]) > s_threshold:  
        
            # evaluate phi_dot from interpolant
            phi_dot = interp_phi_prime([y, x, phi])[0]
        
            # compute (x', y') = (cos(phi), sin(phi)) 
            x_dot = cos(phi)
            y_dot = sin(phi)
    
            # normalize ODE
            norm = sqrt(1+phi_dot**2)
            
            RHS = [x_dot/norm, y_dot/norm, phi_dot/norm]
    
            return RHS

        # if trajectory close to singularity --> stop integration
        else:
            return [0, 0, 0]
   
    # if trajectory outside domain --> stop integration
    else:
        
        return [0, 0, 0]